# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

The most reliable way to do this involves thinking a bit differently about your prompts: just 1 system prompt and 1 user prompt each time, and in the user prompt list the full conversation so far.

Something like:

```python
system_prompt = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie.
"""

user_prompt = f"""
You are Alex, in conversation with Blake and Charlie.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Alex.
"""
```

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>

In [ ]:
# ----------------------------------
# This is a code of a three-way between Alex (ChatGPT 4.1 Mini), Blake (Claude 3.5 Haiku) and Charlie (Deepseek)
# who are cramming for the dreaded Engineering Law test tomorrow.
# They are now deciding what to order from a local pizzeria for a late night study snack.
# ----------------------------------

import os
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

# ----------------------------------
# Setup
# ----------------------------------

load_dotenv(override=True)

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
deepseek_client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com"
)

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
deepseek_model = "deepseek-chat"

# ----------------------------------
# System Prompts
# ----------------------------------

ALEX_SYSTEM = """
You are Alex.
You are highly argumentative, contrarian, and snarky.
You challenge everything said in the conversation, even when you secretly agree.
You enjoy poking holes in logic and tone.
"""

BLAKE_SYSTEM = """
You are Blake.
You are thoughtful, emotionally intelligent, and diplomatic.
You try to mediate disagreements, explain nuance, and find common ground.
You sometimes sound mildly exasperated when Alex refuses to engage in good faith.
"""

CHARLIE_SYSTEM = """
You are Charlie.
You are witty, playful, and slightly chaotic.
You enjoy unexpected observations, jokes, metaphors, and sideways commentary.
You are here to make the conversation more entertaining, not to win arguments.
"""

# ----------------------------------
# Topic Seed
# ----------------------------------

conversation = """
Scenario:
Alex, Blake, and Charlie are part of a late-night study group.
They are hungry and trying to decide what pizza to order for everyone.

Blake: Okay, quick decision please. What pizza do we order so we can get back to studying?
"""

# ----------------------------------
# Model Call Functions
# ----------------------------------
import re

def strip_speaker_labels(text):
    return re.sub(r"^(Alex|Blake|Charlie)(\s*\(.*?\))?:\s*", "", text).strip()
    
def call_gpt(conversation):
    prompt = f"""
You are Alex, in conversation with Blake and Charlie.

Conversation so far:
{conversation}

Constraints:
- Max 256 characters
- React to the most recent speaker
- Stay in character

Respond as Alex.
"""
    response = openai_client.chat.completions.create(
        model=gpt_model,
        messages=[
            {"role": "system", "content": ALEX_SYSTEM},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()


def call_claude(conversation):
    prompt = f"""
You are Blake, in conversation with Alex and Charlie.

Conversation so far:
{conversation}

Constraints:
- Max 256 characters
- React to the most recent speaker
- Stay in character

Respond as Blake.
"""
    response = anthropic_client.messages.create(
        model=claude_model,
        max_tokens=300,
        system=BLAKE_SYSTEM,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.content[0].text.strip()


def call_deepseek(conversation):
    prompt = f"""
You are Charlie, in conversation with Alex and Blake.

Conversation so far:
{conversation}

Constraints:
- Max 256 characters
- React to the most recent speaker
- Stay in character

Respond as Charlie.
"""
    response = deepseek_client.chat.completions.create(
        model=deepseek_model,
        messages=[
            {"role": "system", "content": CHARLIE_SYSTEM},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# ----------------------------------
# Run 5 Turns Each (15 Total Messages)
# ----------------------------------

for turn in range(1, 6):
    alex = call_gpt(conversation)
    conversation += f"\n**Alex (Turn {turn}):** {strip_speaker_labels(alex)}\n"

    blake = call_claude(conversation)
    conversation += f"\n**Blake (Turn {turn}):** {strip_speaker_labels(blake)}\n"

    charlie = call_deepseek(conversation)
    conversation += f"\n**Charlie (Turn {turn}):** {strip_speaker_labels(charlie)}\n"

# ----------------------------------
# Display Final Conversation
# ----------------------------------

display(Markdown(conversation))